In [32]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [33]:
params = torch.load("./model_weights.pt",map_location="cpu")

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=16, pin_memory=True)

testset = datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=16, pin_memory=True)
dataset_sizes = {'train':len(trainset), 'test':len(testset)}


In [34]:
with torch.no_grad():
  # with open("inputs.txt", "w") as in_file:
    num_inputs = 10

    cur_inp = 0
    for data in test_loader:
      inputs, labels  = data

      channels = inputs.shape[1]

      groups = (channels//8)
      if channels%8 != 0:
        groups+=1

      rows = inputs.shape[2]
      cols = inputs.shape[3]

      rem_channels = (8*groups) - channels

      inputs = Variable(inputs.to("cpu"))

      quant_in = torch.quantize_per_tensor(inputs[0], params["quant.scale"].item(), params["quant.zero_point"].item(), torch.quint8).int_repr()
      # if cur_inp<(num_inputs-1):
      #   cur_inp+=1
      #   continue
      filename = "inputs"+str(cur_inp+1)+".txt"
      in_file = open(filename, 'w')
      for row in range(rows):
        for col in range(cols):
          for chn in range(channels):
            in_file.write("{}\n".format(quant_in[chn][row][col]))
          for rem_chn in range(rem_channels):
            in_file.write("0\n")
      cur_inp +=1
      in_file.close()
      if cur_inp>=num_inputs:
        break

        # break
